In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, log_loss
import operator
import json
from IPython import display
import os
import warnings

np.random.seed(0)
warnings.filterwarnings("ignore")
THRESHOLD = 4

Task: To predict whether the user likes the mobile phone or not. <br>
Assumption: If the average rating of mobile >= threshold, then the user likes it, otherwise not.

<b>Missing values:</b><br>
'Also Known As'(459),'Applications'(421),'Audio Features'(437),'Bezel-less display'(266),'Browser'(449),'Build Material'(338),'Co-Processor'(451),'Display Colour'(457),'Mobile High-Definition Link(MHL)'(472),'Music'(447)
'Email','Fingerprint Sensor Position'(174),'Games'(446),'HDMI'(454),'Heart Rate Monitor'(467),'IRIS Scanner'(467),
'Optical Image Stabilisation'(219),'Other Facilities'(444),'Phone Book'(444),'Physical Aperture'(87),'Quick Charging'(122),'Ring Tone'(444),'Ruggedness'(430),SAR Value(315),'SIM 3'(472),'SMS'(470)', 'Screen Protection'(229),'Screen to Body Ratio (claimed by the brand)'(428),'Sensor'(242),'Software Based Aperture'(473),
'Special Features'(459),'Standby time'(334),'Stylus'(473),'TalkTime'(259), 'USB Type-C'(374),'Video Player'(456),
'Video Recording Features'(458),'Waterproof'(398),'Wireless Charging','USB OTG Support'(159), 'Video ,'Recording'(113),'Java'(471),'Browser'(448)

<b>Very low variance:</b><br>
'Architecture'(most entries are 64-bit),'Audio Jack','GPS','Loudspeaker','Network','Network Support','Other Sensors'(28),'SIM Size', 'VoLTE'


<b>Multivalued:</b><br>
'Colours','Custom UI','Model'(1),'Other Sensors','Launch Date'

<b>Not important:</b><br>
'Bluetooth', 'Settings'(75),'Wi-Fi','Wi-Fi Features'

<b>Doubtful:</b><br>
'Aspect Ratio','Autofocus','Brand','Camera Features','Fingerprint Sensor'(very few entries are missing),
'Fingerprint Sensor Position', 'Graphics'(multivalued),'Image resolution'(multivalued),'SIM Size','Sim Slot(s)', 'User Available Storage', 'SIM 1', 'SIM 2','Shooting Modes', 'Touch Screen'(24), 'USB Connectivity'
    
<b>To check:</b><br>
'Display Type','Expandable Memory','FM Radio'

<b>High Correlation with other features</b><br>
'SIM Slot(s)' high correlation with SIM1
'Weight' has high high correlation with capacity , screen-to-body ratio
'Height' - screen size is also there
    
<b>Given a mobile, we can't directly get these features</b><br>
'Rating Count', 'Review Count'

<b>Keeping:</b><br>
'Capacity','Flash'(17),'Height'(22),'Internal Memory'(20, require cleaning),'Operating System'(25, require cleaning), 'Pixel Density'(1, clean it),'Processor'(22, clean it), 'RAM'(17, clean), 'Rating','Resolution'(cleaning), 'Screen Resolution','Screen Size', 'Thickness'(22), 'Type','User Replaceable','Weight'(cleaning),'Sim Size'(), 'Other Sensors'(28), 'Screen to Body Ratio (calculated)','Width',


In [2]:
# read data from file
train = pd.read_csv("../input/train.csv") 
test = pd.read_csv("../input/test.csv")
#print(train['Rating'])
# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 355
Number of features in train: 99
Number of data points in test: 119
Number of features in test: 98


In [3]:
def data_clean(data):
    
    # Let's first remove all missing value features
    columns_to_remove = ['Also Known As','Applications','Audio Features','Bezel-less display'
                         'Browser','Build Material','Co-Processor','Browser'
                         'Display Colour','Mobile High-Definition Link(MHL)',
                         'Music', 'Email','Fingerprint Sensor Position',
                         'Games','HDMI','Heart Rate Monitor','IRIS Scanner', 
                         'Optical Image Stabilisation','Other Facilities',
                         'Phone Book','Physical Aperture','Quick Charging',
                         'Ring Tone','Ruggedness','SAR Value','SIM 3','SMS',
                         'Screen Protection','Screen to Body Ratio (claimed by the brand)',
                         'Sensor','Software Based Aperture', 'Special Features',
                         'Standby time','Stylus','TalkTime', 'USB Type-C',
                         'Video Player', 'Video Recording Features','Waterproof',
                         'Wireless Charging','USB OTG Support', 'Video Recording','Java']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    #Features having very low variance 
    columns_to_remove = ['Architecture','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    # Multivalued:
    columns_to_remove = ['Architecture','Launch Date','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE', 'Custom UI']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    # Not much important
    columns_to_remove = ['Bluetooth', 'Settings','Wi-Fi','Wi-Fi Features']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    
    return data

# Removing features

In [4]:
train = data_clean(train)
test = data_clean(test)

removing all those data points in which more than 15 features are missing 

In [5]:
train = train[(train.isnull().sum(axis=1) <= 15)]
# You shouldn't remove data points from test set
#test = test[(test.isnull().sum(axis=1) <= 15)]

In [6]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 47
Number of data points in test: 119
Number of features in test: 46


# Filling Missing values

In [7]:
def for_integer(test):
    try:
        test = test.strip()
        return int(test.split(' ')[0])
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass

def for_string(test):
    try:
        test = test.strip()
        return (test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

def for_float(test):
    try:
        test = test.strip()
        return float(test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

    
def for_Internal_Memory(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[1] == 'GB':
            return int(test[0])
        if test[1] == 'MB':
#             print("here")
            return (int(test[0]) * 0.001)
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass
    
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass


In [8]:
def data_clean_2(x):
    data = x.copy()
    
    data['Capacity'] = data['Capacity'].apply(for_integer)

    data['Height'] = data['Height'].apply(for_float)
    data['Height'] = data['Height'].fillna(data['Height'].mean())

    data['Internal Memory'] = data['Internal Memory'].apply(for_Internal_Memory)

    data['Pixel Density'] = data['Pixel Density'].apply(for_integer)

    data['Internal Memory'] = data['Internal Memory'].fillna(data['Internal Memory'].median())
    data['Internal Memory'] = data['Internal Memory'].astype(int)

    data['RAM'] = data['RAM'].apply(for_integer)
    data['RAM'] = data['RAM'].fillna(data['RAM'].median())
    data['RAM'] = data['RAM'].astype(int)

    data['Resolution'] = data['Resolution'].apply(for_integer)
    data['Resolution'] = data['Resolution'].fillna(data['Resolution'].median())
    data['Resolution'] = data['Resolution'].astype(int)

    data['Screen Size'] = data['Screen Size'].apply(for_float)

    data['Thickness'] = data['Thickness'].apply(for_float)
    data['Thickness'] = data['Thickness'].fillna(data['Thickness'].mean())
    data['Thickness'] = data['Thickness'].round(2)

    data['Type'] = data['Type'].fillna('Li-Polymer')

    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].apply(for_float)
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].fillna(data['Screen to Body Ratio (calculated)'].mean())
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].round(2)

    data['Width'] = data['Width'].apply(for_float)
    data['Width'] = data['Width'].fillna(data['Width'].mean())
    data['Width'] = data['Width'].round(2)

    data['Flash'][data['Flash'].isna() == True] = "Other"

    data['User Replaceable'][data['User Replaceable'].isna() == True] = "Other"

    data['Num_cores'] = data['Processor'].apply(for_string)
    data['Num_cores'][data['Num_cores'].isna() == True] = "Other"


    data['Processor_frequency'] = data['Processor'].apply(find_freq)
    #because there is one entry with 208MHz values, to convert it to GHz
    data['Processor_frequency'][data['Processor_frequency'] > 200] = 0.208
    data['Processor_frequency'] = data['Processor_frequency'].fillna(data['Processor_frequency'].mean())
    data['Processor_frequency'] = data['Processor_frequency'].round(2)

    data['Camera Features'][data['Camera Features'].isna() == True] = "Other"

    #simplifyig Operating System to os_name for simplicity
    data['os_name'] = data['Operating System'].apply(for_string)
    data['os_name'][data['os_name'].isna() == True] = "Other"

    data['Sim1'] = data['SIM 1'].apply(for_string)

    data['SIM Size'][data['SIM Size'].isna() == True] = "Other"

    data['Image Resolution'][data['Image Resolution'].isna() == True] = "Other"

    data['Fingerprint Sensor'][data['Fingerprint Sensor'].isna() == True] = "Other"

    data['Expandable Memory'][data['Expandable Memory'].isna() == True] = "No"

    data['Weight'] = data['Weight'].apply(for_integer)
    data['Weight'] = data['Weight'].fillna(data['Weight'].mean())
    data['Weight'] = data['Weight'].astype(int)

    data['SIM 2'] = data['SIM 2'].apply(for_string)
    data['SIM 2'][data['SIM 2'].isna() == True] = "Other"
    
    return data

In [9]:
train = data_clean_2(train)
test = data_clean_2(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 51
Number of data points in test: 119
Number of features in test: 50


Not very important feature

In [10]:
def data_clean_3(x):
    
    data = x.copy()

    columns_to_remove = ['User Available Storage','SIM Size','Chipset','Processor','Autofocus','Aspect Ratio','Touch Screen',
                        'Bezel-less display','Operating System','SIM 1','USB Connectivity','Other Sensors','Graphics','FM Radio',
                        'NFC','Shooting Modes','Browser','Display Colour' ]

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = [ 'Screen Resolution','User Replaceable','Camera Features',
                        'Thickness', 'Display Type']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = ['Fingerprint Sensor', 'Flash', 'Rating Count', 'Review Count','Image Resolution','Type','Expandable Memory',\
                        'Colours','Width','Model']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    return data

In [11]:
train = data_clean_3(train)
test = data_clean_3(test)
print(test.columns)
# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Index(['RAM', 'Processor_frequency', 'Brand', 'Num_cores',
       'Screen to Body Ratio (calculated)', 'SIM Slot(s)', 'PhoneId',
       'Screen Size', 'SIM 2', 'Resolution', 'Sim1', 'os_name',
       'Pixel Density', 'Internal Memory', 'Height', 'Capacity', 'Weight'],
      dtype='object')
Number of data points in train: 341
Number of features in train: 18
Number of data points in test: 119
Number of features in test: 17


In [12]:
# one hot encoding

train_ids = train['PhoneId']
test_ids = test['PhoneId']

cols = list(test.columns)
print(cols)
cols.remove('PhoneId')
print(cols)
cols.insert(0, 'PhoneId')
print("Train columns are",train.columns)
#combined=train.drop('Rating', axis=1)[cols]
combined = pd.concat([train.drop('Rating', axis=1)[cols], test[cols]])
print(combined.shape)
print(combined.columns)

combined = pd.get_dummies(combined)
print(combined.shape)
print(combined.columns)

train_new = combined[combined['PhoneId'].isin(train_ids)]
test_new = combined[combined['PhoneId'].isin(test_ids)]

['RAM', 'Processor_frequency', 'Brand', 'Num_cores', 'Screen to Body Ratio (calculated)', 'SIM Slot(s)', 'PhoneId', 'Screen Size', 'SIM 2', 'Resolution', 'Sim1', 'os_name', 'Pixel Density', 'Internal Memory', 'Height', 'Capacity', 'Weight']
['RAM', 'Processor_frequency', 'Brand', 'Num_cores', 'Screen to Body Ratio (calculated)', 'SIM Slot(s)', 'Screen Size', 'SIM 2', 'Resolution', 'Sim1', 'os_name', 'Pixel Density', 'Internal Memory', 'Height', 'Capacity', 'Weight']
Train columns are Index(['RAM', 'Processor_frequency', 'Brand', 'Num_cores',
       'Screen to Body Ratio (calculated)', 'SIM Slot(s)', 'PhoneId',
       'Screen Size', 'SIM 2', 'Resolution', 'Sim1', 'os_name',
       'Pixel Density', 'Internal Memory', 'Height', 'Rating', 'Capacity',
       'Weight'],
      dtype='object')
(460, 17)
Index(['PhoneId', 'RAM', 'Processor_frequency', 'Brand', 'Num_cores',
       'Screen to Body Ratio (calculated)', 'SIM Slot(s)', 'Screen Size',
       'SIM 2', 'Resolution', 'Sim1', 'os_name', 

In [13]:
train_new = train_new.merge(train[['PhoneId', 'Rating']], on='PhoneId')

In [14]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train_new.shape[0])
print("Number of features in train: %d" % train_new.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test_new.shape[0])
print("Number of features in test: %d" % test_new.shape[1])

Number of data points in train: 341
Number of features in train: 88
Number of data points in test: 119
Number of features in test: 87


In [15]:
#list(set(train_new.columns) - set(test_new.columns))
#print(test_new.columns)
train_new.head()

,PhoneId,RAM,Processor_frequency,Screen to Body Ratio (calculated),Screen Size,Resolution,Pixel Density,Internal Memory,Height,Capacity,Weight,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,Brand_Gionee,Brand_Google,Brand_HTC,Brand_Honor,Brand_Huawei,Brand_InFocus,Brand_Infinix,Brand_Intex,Brand_Itel,Brand_Jivi,Brand_Karbonn,Brand_LG,Brand_Lava,Brand_LeEco,Brand_Lenovo,Brand_Lephone,Brand_Lyf,Brand_Meizu,Brand_Micromax,Brand_Mobiistar,Brand_Moto,...,Brand_Realme,Brand_Reliance,Brand_Samsung,Brand_Sony,Brand_Spice,Brand_Tecno,Brand_Ulefone,Brand_VOTO,Brand_Vivo,Brand_Xiaomi,Brand_Xiaomi Poco,Brand_Yu,Brand_iVooMi,Num_cores_312,Num_cores_Deca,Num_cores_Dual,Num_cores_Hexa,Num_cores_Octa,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other,Sim1_2G,Sim1_3G,Sim1_4G,os_name_Android,os_name_Blackberry,os_name_KAI,os_name_Nokia,os_name_Other,os_name_Tizen,os_name_iOS,Rating
0,0,4,1.8,80.68,6.26,20,403,64,157.9,4000,182,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,4.5
1,1,3,1.8,80.85,6.20,8,271,32,156.2,4230,168,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,4.5
2,2,3,2.1,83.68,6.30,25,409,32,157.0,3500,168,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,4.4
3,4,4,2.2,74.78,6.00,24,411,64,159.8,3300,169,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,4.3
4,5,4,2.2,84.23,6.50,16,396,64,160.4,3750,175,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,4.4


In [16]:
test_new.head()

,PhoneId,RAM,Processor_frequency,Screen to Body Ratio (calculated),Screen Size,Resolution,Pixel Density,Internal Memory,Height,Capacity,Weight,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Billion,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Do,Brand_Gionee,Brand_Google,Brand_HTC,Brand_Honor,Brand_Huawei,Brand_InFocus,Brand_Infinix,Brand_Intex,Brand_Itel,Brand_Jivi,Brand_Karbonn,Brand_LG,Brand_Lava,Brand_LeEco,Brand_Lenovo,Brand_Lephone,Brand_Lyf,Brand_Meizu,Brand_Micromax,Brand_Mobiistar,Brand_Moto,...,Brand_Razer,Brand_Realme,Brand_Reliance,Brand_Samsung,Brand_Sony,Brand_Spice,Brand_Tecno,Brand_Ulefone,Brand_VOTO,Brand_Vivo,Brand_Xiaomi,Brand_Xiaomi Poco,Brand_Yu,Brand_iVooMi,Num_cores_312,Num_cores_Deca,Num_cores_Dual,Num_cores_Hexa,Num_cores_Octa,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other,Sim1_2G,Sim1_3G,Sim1_4G,os_name_Android,os_name_Blackberry,os_name_KAI,os_name_Nokia,os_name_Other,os_name_Tizen,os_name_iOS
0,3,2,1.8,80.85,6.20,5,271,16,156.2,4230,168,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0
1,11,4,1.8,81.60,6.20,12,402,64,156.0,5000,205,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0
2,13,6,2.0,83.84,6.30,25,409,64,156.7,3500,169,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0
3,16,4,2.0,83.84,6.30,16,409,64,156.7,3500,169,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0
4,19,4,1.8,77.43,5.99,20,403,64,158.6,4000,181,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0


## Solution

In [17]:
#train_new=train_new.drop(train_new[train_new.Rating == 4 ].index)
#train_new=train_new.drop(train_new[i for i in train_new.Rating if i >= 3.5 and i < 4].index)
train_new['Rating']=train_new['Rating'].map(lambda x : 1 if x >=4  else 0)
#train_new['Rating']=train_new['Rating'].map(lambda x : 1 if x > 4  else 0)
train_new=train_new.drop('PhoneId',axis=1)
phone_id=test_new['PhoneId']
test_new=test_new.drop('PhoneId',axis=1)
phone_id=pd.DataFrame(phone_id)

## Variance Calculation

In [18]:
train_normalise=pd.DataFrame(columns=train_new.columns)
for i in train_new.columns:
  if train_new[i].min()==train_new[i].max():
    train_normalise[i]=0
  else:
    train_normalise[i]=(train_new[i] - train_new[i].min())/(train_new[i].max()-train_new[i].min())
    
test_normalise=pd.DataFrame(columns=test_new.columns)
for i in test_new.columns:
  if test_new[i].min()==test_new[i].max():
    test_normalise[i]=0
  else:
    test_normalise[i]=(test_new[i] - test_new[i].min())/(test_new[i].max()-test_new[i].min())
 
Y=train_normalise['Rating']
train_normalise=train_normalise.drop('Rating',axis=1)
X=train_normalise
train_variance=train_normalise.var(axis=0)
train_variance=pd.DataFrame(train_variance)
train_variance=train_variance.T
#normalise.head()
train_variance.head()
#avg_variance=variance.mean(axis=1)
#print(avg_variance)
drop_columns=[]
for x in train_variance.columns:
  if train_variance[x][0]==0:
    drop_columns.append(x)
print(train_normalise.shape)
print(train_variance)
print(drop_columns)

(341, 86)
      RAM  Processor_frequency     ...       os_name_Tizen  os_name_iOS
0  0.0144             0.062945     ...            0.002933     0.052769

[1 rows x 86 columns]
['Brand_Billion', 'Brand_Do', 'Brand_Spice']


## Delete the columns with less variance from the average variance

In [19]:
for i in drop_columns:
    X=X.drop(i,axis=1)
    test_normalise=test_normalise.drop(i,axis=1)
    train_variance=train_variance.drop(i,axis=1)
print(train_normalise.shape)
print(test_normalise.shape)
print(train_variance)
X.head()

(341, 86)
(119, 83)
      RAM  Processor_frequency     ...       os_name_Tizen  os_name_iOS
0  0.0144             0.062945     ...            0.002933     0.052769

[1 rows x 83 columns]


,RAM,Processor_frequency,Screen to Body Ratio (calculated),Screen Size,Resolution,Pixel Density,Internal Memory,Height,Capacity,Weight,Brand_10.or,Brand_Apple,Brand_Asus,Brand_Blackberry,Brand_Comio,Brand_Coolpad,Brand_Gionee,Brand_Google,Brand_HTC,Brand_Honor,Brand_Huawei,Brand_InFocus,Brand_Infinix,Brand_Intex,Brand_Itel,Brand_Jivi,Brand_Karbonn,Brand_LG,Brand_Lava,Brand_LeEco,Brand_Lenovo,Brand_Lephone,Brand_Lyf,Brand_Meizu,Brand_Micromax,Brand_Mobiistar,Brand_Moto,Brand_Motorola,Brand_Nokia,Brand_Nubia,...,Brand_Panasonic,Brand_Razer,Brand_Realme,Brand_Reliance,Brand_Samsung,Brand_Sony,Brand_Tecno,Brand_Ulefone,Brand_VOTO,Brand_Vivo,Brand_Xiaomi,Brand_Xiaomi Poco,Brand_Yu,Brand_iVooMi,Num_cores_312,Num_cores_Deca,Num_cores_Dual,Num_cores_Hexa,Num_cores_Octa,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other,Sim1_2G,Sim1_3G,Sim1_4G,os_name_Android,os_name_Blackberry,os_name_KAI,os_name_Nokia,os_name_Other,os_name_Tizen,os_name_iOS
0,0.005871,0.444444,0.891040,0.921241,0.782609,0.600462,0.1250,0.723077,0.248748,0.455882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.003914,0.444444,0.893693,0.906921,0.260870,0.295612,0.0625,0.694017,0.267947,0.404412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.003914,0.611111,0.937871,0.930788,1.000000,0.614319,0.0625,0.707692,0.207012,0.404412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.005871,0.666667,0.798938,0.859189,0.956522,0.618938,0.1250,0.755556,0.190317,0.408088,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.005871,0.666667,0.946456,0.978520,0.608696,0.584296,0.1250,0.765812,0.227880,0.430147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Threshold for Binarization

In [20]:
#train_mean=train_new.groupby('Rating').mean()
#print(train_mean)
#train_new.head()

print("X=",X.head())


Y=pd.DataFrame(Y)

'''index=['thres','value']
columns=X.columns
mean_data=pd.DataFrame(index=index,columns=columns)

for i in X.columns:
  mean_data[i]['thres']=(train_mean[i][0]+train_mean[i][1])/2
  if train_mean[i][0]>train_mean[i][1]:
    mean_data[i]['value']=0
  else:
    mean_data[i]['value']=1
print(mean_data.head())'''

X=         RAM  Processor_frequency     ...       os_name_Tizen  os_name_iOS
0  0.005871             0.444444     ...                 0.0          0.0
1  0.003914             0.444444     ...                 0.0          0.0
2  0.003914             0.611111     ...                 0.0          0.0
3  0.005871             0.666667     ...                 0.0          0.0
4  0.005871             0.666667     ...                 0.0          0.0

[5 rows x 83 columns]


"index=['thres','value']\ncolumns=X.columns\nmean_data=pd.DataFrame(index=index,columns=columns)\n\nfor i in X.columns:\n  mean_data[i]['thres']=(train_mean[i][0]+train_mean[i][1])/2\n  if train_mean[i][0]>train_mean[i][1]:\n    mean_data[i]['value']=0\n  else:\n    mean_data[i]['value']=1\nprint(mean_data.head())"

## Binarization of Input

In [21]:
'''for i in X.columns:
  X[i]=X[i].map(lambda x : mean_data[i]['value'] if x>=mean_data[i]['thres'] else int(not mean_data[i]['value']))
  
for j in test_new.columns:
  test_new[j]=test_new[j].map(lambda x : mean_data[j]['value'] if x>=mean_data[j]['thres'] else int(not mean_data[j]['value']))

print(type(X))
print(test_new)'''

"for i in X.columns:\n  X[i]=X[i].map(lambda x : mean_data[i]['value'] if x>=mean_data[i]['thres'] else int(not mean_data[i]['value']))\n  \nfor j in test_new.columns:\n  test_new[j]=test_new[j].map(lambda x : mean_data[j]['value'] if x>=mean_data[j]['thres'] else int(not mean_data[j]['value']))\n\nprint(type(X))\nprint(test_new)"

## Train - Test Split

In [22]:
train_mean=X.groupby(Y['Rating']).mean()
print(train_mean)
weight=np.zeros(X.shape[1])
for i,j in zip(X.columns,range(0,X.shape[1])):
    weight[j]=(1 if train_mean[i][0]<train_mean[i][1] else -1)
weight=weight*train_variance
print(weight)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.1, stratify=Y, random_state=1)

             RAM  Processor_frequency     ...       os_name_Tizen  os_name_iOS
Rating                                    ...                                 
0.0     0.051945             0.315804     ...            0.009709     0.000000
1.0     0.005896             0.494188     ...            0.000000     0.079832

[2 rows x 83 columns]
      RAM  Processor_frequency     ...       os_name_Tizen  os_name_iOS
0 -0.0144             0.062945     ...           -0.002933     0.052769

[1 rows x 83 columns]


> ## Perceptron

In [23]:
class Perceptron:
  
  def __init__(self):
    self.w = None
    self.b = None
    
  def model(self,x):
    return 1 if (np.dot(self.w,x)>=self.b) else 0
  
  def predict(self,X):
    y_pred = []
    for x in X:
      y_pred.append(self.model(x))
    return np.asarray(y_pred)
  
  def fit(self,X,Y,weight,epocs=5):
    self.w=weight
    self.b=0
    for i in range(epocs):
      for x,y in zip(X,Y):
        y_pred=self.model(x)
        if y == 0 and y_pred==1:
          self.w=self.w-(0.8*x)
          self.b=self.b-0.8
        elif y==1 and y_pred==0:
          self.w=self.w+(0.8*x)
          self.b=self.b+0.8

In [24]:
print(Y_train.head())
X_train=X_train.values
Y_train=Y_train.values
#print(Y_train)
X_test=X_test.values
Y_test=Y_test.values
test_data=test_normalise.values
perceptron=Perceptron()
max=0
for i in range(1,200):
    perceptron.fit(X_train,Y_train,weight,i)
    test_pred=perceptron.predict(X_test)
    if max<accuracy_score(test_pred,Y_test):
        max=accuracy_score(test_pred,Y_test)
        temp=i
perceptron.fit(X_train,Y_train,weight,temp)
test_pred=perceptron.predict(X_test)
print("max=",max)
print("temp=",temp)
pred_out=perceptron.predict(test_data)
print(pred_out)
pred_out=pd.DataFrame(pred_out)
pred_out.columns=['Class']
test_new=pd.DataFrame(test_new)
#pred_out=pred_out.values
#print(pred_out)
print(type(test_normalise))
print(accuracy_score(test_pred,Y_test))

     Rating
248     0.0
58      1.0
24      1.0
200     1.0
182     0.0
max= 0.8
temp= 7
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 0 1 0 1 1 0 1 1 0 1 1 0 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 1 0
 1 0 1 0 1 1 1 0]
<class 'pandas.core.frame.DataFrame'>
0.8


In [25]:
submission=pd.DataFrame({'PhoneId':phone_id['PhoneId'], 'Class':pred_out['Class']})
submission = submission[['PhoneId', 'Class']]
print(submission.head())
submission.to_csv("submission.csv", index=False)
print(submission.shape)

   PhoneId  Class
0        3      1
1       11      1
2       13      1
3       16      1
4       19      1
(119, 2)


In [26]:
"""
WRITE YOUR MODELLING CODE HERE
"""

'\nWRITE YOUR MODELLING CODE HERE\n'